In [1]:
%env CUDA_VISIBLE_DEVICES=6

import math

import torch
from linear import HiggsLinear

env: CUDA_VISIBLE_DEVICES=6


In [2]:
SIZE = 8192
HADAMARD_SIZE = 1024
GRID_DIM = 2

layer = HiggsLinear(
    SIZE, SIZE,
    GRID_DIM, 256,
    device="cuda",
    dtype=torch.float16,
)

In [3]:
input = torch.rand(1, 1, SIZE, device="cuda", dtype=torch.float16)

layer(input)

tensor([[[ 40.5312,  36.6562, -18.4844,  ...,  -0.8687, -15.7422, -27.4219]]],
       device='cuda:0', dtype=torch.float16)

In [4]:
from aqlm.utils import unpack_int_data, _dequantize_weight

codebooks = torch.load(f"../grids/EDEN{GRID_DIM}-256.pt").half()

weight = _dequantize_weight(
    unpack_int_data(layer.codes, 8)[:,:,None],
    codebooks[None,:,None,:],
    # scales=layer.scales,
).reshape(-1, SIZE//HADAMARD_SIZE, HADAMARD_SIZE) * layer.scales[...,None]

weight = weight.reshape(-1, SIZE)

In [5]:
weight.shape

torch.Size([8192, 8192])

In [6]:
from fast_hadamard_transform import hadamard_transform

hadamard_transform(input.reshape(-1, HADAMARD_SIZE), scale=1/math.sqrt(HADAMARD_SIZE)).reshape(1, -1) @ weight.T

tensor([[ 40.5625,  36.6250, -18.5156,  ...,  -0.8691, -15.8047, -27.4219]],
       device='cuda:0', dtype=torch.float16)

In [7]:
%%time

with torch.inference_mode():
    for i in range(1000):
        layer(input)
        torch.cuda.synchronize()

CPU times: user 212 ms, sys: 1.01 ms, total: 213 ms
Wall time: 211 ms


In [8]:
%%time

with torch.inference_mode():
    for i in range(1000):
        torch.nn.functional.linear(input, weight)
        torch.cuda.synchronize()

CPU times: user 183 ms, sys: 1.19 ms, total: 184 ms
Wall time: 182 ms
